# Opening the Blackbox: Collision Attacks on Round-Reduced Tip5, Tip4, Tip4' and Monolith

In [1]:
# Import Hash functions
load("Tip5.sage")
load("Monolith.sage")

Mono64 = Monolith(n=64, eta=32, t=12, u=4, sis=[8, 8, 8, 8, 8, 8, 8, 8], r=8, c=4, d=4, N=2) # 2 rounds
Mono31 = Monolith(n=31, eta=None, t=24, u=8, sis=[7, 8, 8, 8], r=16, c=8, d=8, N=2) # 2 rounds

In [2]:
# Utility functions
load("utils/magma.sage")
load("utils/polys.sage")
load("utils/complexities.sage")

In [3]:
macaulay_bound = lambda eqs : sum([f.degree()-1 for f in eqs]) + 1
bezout_bound = lambda eqs : prod([f.degree() for f in eqs])

## Section 3: Cryptanalysis of Tip5 and Monolith Independent of the S-box

In [4]:
def mono_simple_ca(mono, innerCollison, debug=False, ge=True, writeFile=False):
    '''
    mono  Monolith instance
    innerCollison    
          If true: target collision in the last c truncated words.
          If false: target collision in the first d truncated words (hash value).
    '''    
    # ----------------------------------------
    # Variables
    # ----------------------------------------
    varnames = flatten([[f'x{i}',f'y{i}'] for i in range(mono.r)])
    P = PolynomialRing(mono.F, varnames, order='degrevlex')
    if debug:
        print(P)
    
    # Input vectors to first round
    i0 = vector(P, mono.t)
    I0 = vector(P, mono.t)
    
    i0[:mono.r] = [P(f'x{i}') for i in range(mono.r)]
    i0[mono.r:] = [mono.c_value] * mono.c
    
    I0[:mono.r] = [P(f'y{i}') for i in range(mono.r)]
    I0[mono.r:] = [mono.c_value] * mono.c
    
    if debug:
        print("Inputs to permutation")
        print(i0)
        print(I0)
    
    # ----------------------------------------
    # Equations
    # ----------------------------------------
    
    # Input vectors to first BAR layer
    i1 = mono.concrete(i0)
    I1 = mono.concrete(I0)
    
    # Output vectors to first BAR layer
    o1 = i1
    O1 = I1
    
    # (1) Forward: Randomly fix inputs to S-Boxes in first BAR layer
    eqs_r1 = []
    for i in range(mono.u):
        rx = mono.F.random_element()
        ry = mono.F.random_element()
        
        eqs_r1.append(i1[i] - rx)
        eqs_r1.append(I1[i] - ry)
        
        # Fixes S-Box outputs in first BAR layer
        o1[i] = mono.bar(rx)
        O1[i] = mono.bar(ry)
    assert(all([f.degree() == 1 for f in eqs_r1]))
    
    # Input vectors to second BAR layer
    i2 = mono.linear_layer(mono.bricks(o1), r=0)
    I2 = mono.linear_layer(mono.bricks(O1), r=0)
    
    # Input difference in second BAR layer
    Di2 = I2 - i2
    
    # (2) Forward: Ensure zero-difference to S-Box inputs/outputs in second round
    eqs_r2 = []
    for i in range(mono.u):
        eqs_r2.append(Di2[i])
    assert(all([f.degree() == 2 for f in eqs_r2]))
    
    # (3) Forward: Collision (= zero difference) 
    # If innerCollision is true: in inner part (c state words)
    # If innerCollision is false: in hash (d state words)
    
    # Ignore first u entries for nonlinear layer. 
    # The first u+1 outputs of the nonlinear layer will not be correct, but all differences will be correct.
    i2[:mono.u] = [0]*mono.u  
    I2[:mono.u] = [0]*mono.u
    
    o2 = mono.nonlinear_layer(i2)
    O2 = mono.nonlinear_layer(I2)
    
    # Output difference after BARS + BRICKS
    Do2 = O2 - o2
    assert(all([Do2[i] == 0 for i in range(mono.u)]))
    
    if debug:
        print(f"Degrees output differences 2nd non-linear layer:\n", [f.degree() for f in Do2])
    
    # Output differences (without RC addition, because of difference)
    Do = mono.concrete(Do2)
    if debug:
        print(f"Degrees output differences:\n", [f.degree() for f in Do])
    assert(all([f.degree() == 4 for f in Do]))
    
    if innerCollison:
        eqs_col = [Do[i] for i in range(mono.r, mono.t)] # zero difference in inner part
        assert(len(eqs_col) == mono.c)
    else:
        eqs_col = [Do[i] for i in range(mono.d)] # zero hash difference
    
    
    print(f"R1: {len(eqs_r1)} equations of degree {1} in {Sequence(eqs_r1).nvariables()} variables", flush=True)
    print(f"R2: {len(eqs_r2)} equations of degree {2} in {Sequence(eqs_r2).nvariables()} variables", flush=True)
    print(f"Do: {len(eqs_col)} equations of degree {4} in {Sequence(eqs_col).nvariables()} variables", flush=True)
    eqs = eqs_r1 + eqs_r2 + eqs_col
    
    # ----------------------------------------
    # Gaussian elimination
    # ----------------------------------------
    
    if ge:
        # Perform Gaussian Elimination
        print('-'*80 + "\nPerform Gaussian Elimination\n" + '-'*80)
        eqs, sub_lin = gaussian_elimination(eqs)
        #print(sub_lin)
        fixed_elems = [sub_lin]
        mdir = "magma/GE"
    else:
        fixed_elems = []
        mdir = "magma"
    
    print_stats(eqs)
    
    # ----------------------------------------
    # Write to magma file
    # ----------------------------------------
    if writeFile:
        gen_magma_file_mono(mono, eqs, fixed_elems=fixed_elems, attacktype=f"{'i' if innerCollison else ''}ca_{mono.name.lower()}", mdir=mdir, replace=False) 
    return eqs

In [5]:
mono_simple_ca(Mono64, innerCollison=True, debug=False, ge=True, writeFile=False);

R1: 8 equations of degree 1 in 16 variables
R2: 4 equations of degree 2 in 16 variables
Do: 4 equations of degree 4 in 16 variables
--------------------------------------------------------------------------------
Perform Gaussian Elimination
--------------------------------------------------------------------------------
Total: 16 equations in 16 variables
8 linear equations in 16 variables
8 non-linear equations in 16 variables
-> 8 non-linear equations in 8 variables
8 equations in 8 variables
--------------------------------------------------------------------------------
Macaulay bound:  17
GB complexity:   20.1w + 0.0 (47.6)
--------------------------------------------------------------------------------
Bézout bound:    2^12
FGLM complexity: 12.0w + 3.0 (39.0)


## Section 5: Two-stage Collisions Attacks on Tip5, Tip4 and Tip4'

#### IO differences for S-box S and powermap T

In [6]:
# IO differences for S-box S and powermap T
load("utils/differences.sage")

In [7]:
din_S, dout_S = din_to_dout_S(tip4, tip4.random_element(), debug=True)
assert(valid_din_dout_S(tip4,din_S,dout_S))

437646084114230198 = [6, 18, 212, 203, 116, 248, 43, 182]
437646084114230198 = [6, 18, 212, 203, 116, 248, 43, 182] -> [70, 1, 155, 203, 110, 68, 39, 103] = 5044484355662292839


In [8]:
din_S, dout_S = din_to_dout_S(tip4, tip4.random_element(), u=1, debug=True)
assert(valid_din_dout_S(tip4,din_S,dout_S))

13864375918500901204 = [192, 104, 40, 112, 246, 185, 241, 84]
84 = [0, 0, 0, 0, 0, 0, 0, 84] -> [0, 0, 0, 0, 0, 0, 0, 90] = 90


In [9]:
din_dout_to_in_S(tip4, din_S, dout_S, debug=True)

Differences U: [0, 0, 0, 0, 0, 0, 0, 84] -> [0, 0, 0, 0, 0, 0, 0, 90]
--------------------------------------------------------------------------------
Concrete input to S-Box
in_S = 3113827563294591303
in_S + din_S = 3113827202517338439
Difference: 18446743708637331457 = 18446743708637331457
R * in_S = 14555221979629808910 = [201, 254, 137, 70, 10, 202, 237, 14]
R * (in_S + din_S) = 14555221979629808994 = [201, 254, 137, 70, 10, 202, 237, 98]
Input difference: 84 = [0, 0, 0, 0, 0, 0, 0, 84]
L(R * in_S) = 11599292033877692193 = [160, 248, 246, 166, 45, 76, 79, 33]
L(R * (in_S + din_S)) = 11599292033877692283 = [160, 248, 246, 166, 45, 76, 79, 123]
Output difference: 90 = [0, 0, 0, 0, 0, 0, 0, 90]
Concrete output to S-Box
S(in_S) = 3583381205908256423
S(in_S + din_S) = 3583380819361199783
Difference: 18446743682867527681 = 18446743682867527681
--------------------------------------------------------------------------------
Concrete input to S-Box
in_S = 3113826949114267975
in_S + din_S =

[14555221979629808910, 14555221979629809053]

### Section 5.1: Stage 1 - Find non-zero differences and valid values

In [10]:
def two_stage_ca_stage1(tip, s1, t1, u=8, debug=1, fixInactiveS=False):
    '''
    tip    Tip5 like instance
    s1     Number of active S-Boxes (first s1 of tip.s S-Boxes active)
    t1     Number of active T-Boxes (first t1 of tip.m-tip.s T-Boxes active)
    u      Number of active chunks in s1-th S-Box
    '''
    t = tip.m - tip.s # Number of T-Boxes
    s0 = tip.s - s1
    
    # First s1 of tip.s S-Boxes active
    assert(s1 <= tip.s)
    # First t1 of tip.m-tip.s T-Boxes active
    assert(t1 <= t)
    # Number of active chunks in first S-Box
    assert(u <= 8)
    
    # ----------------------------------------
    # Variables
    # ----------------------------------------
    
    idx_sdiff = list(range(s1))                # indices for S-Boxes with non-zero IO difference
    idx_tdiff = list(range(tip.s,tip.s + t1))  # indices for T-Boxes with non-zero IO difference
    
    varnames = flatten([[f'Dx{i}', f'Dy{i}'] for i in idx_sdiff] + [[f'Dx{i}', f'Dy{i}'] for i in idx_tdiff])
    P = PolynomialRing(tip.field, varnames, order='degrevlex')
    
    Di2 = vector(P, tip.m) # Input differences to second round non-linear layer
    Do2 = vector(P, tip.m) # Output differences to second round non-linear layer
    
    # For S-boxes, in order to use internals of LUT later, Dx and Dy are the 
    # differences of the words directly entered in composition and decomposition.
    for i in idx_sdiff:
        Di2[i] = tip.R_inv * P(f'Dx{i}')
        Do2[i] = tip.R_inv * P(f'Dy{i}')
    
    for i in idx_tdiff:
        Di2[i] = P(f'Dx{i}')
        Do2[i] = P(f'Dy{i}')
    
    if debug > 0:
        print("Di2 = ", Di2)
        print("Do2 = ", Do2)
    
    # ----------------------------------------
    # Equations
    # ----------------------------------------
    
    # Backward: Conditions on initial capacity part in first round
    Do1 = tip.multiply_mds_matrix(Di2, inv=True)
    sys1_Do1 = [Do1[i] for i in range(tip.r,tip.m)]    
    assert(all([f.degree() == 1 for f in sys1_Do1]))
    
    sys1_Do1_neqs = len(sys1_Do1)
    sys1_Do1_nvars = Sequence(sys1_Do1).nvariables()
    
    if debug > 0:
        print(f"Sys1 (Do1): {sys1_Do1_neqs} equations of degree {1} in {sys1_Do1_nvars} variables", flush=True)
    
    # Forward: No activate S-boxes in third round
    Di3 = tip.multiply_mds_matrix(Do2, inv=False)
    sys2_Di3 = [Di3[i] for i in range(tip.s)]
    assert(all([f.degree() == 1 for f in sys2_Di3]))
    
    sys2_Di3_neqs = len(sys2_Di3)
    sys2_Di3_nvars = Sequence(sys2_Di3).nvariables()
    
    if debug > 0:
        print(f"Sys2 (Di3): {sys2_Di3_neqs} equations of degree {1} in {sys2_Di3_nvars} variables", flush=True)
    
    # ----------------------------------------
    # Filter solutions
    # ----------------------------------------
    
    found_solution = False
    numtries = 0
    
    while not found_solution:
        numtries += 1
        
        sys1_Do1_nfreevars = sys1_Do1_nvars - sys1_Do1_neqs
        assert(sys1_Do1_nfreevars >= 0) # More variables than equations
        
        sys2_Di3_nfreevars = sys2_Di3_nvars - sys2_Di3_neqs
        assert(sys2_Di3_nfreevars >= 0) # More variables than equations
        
        diffs = {} # values for difference
        vals = {}  # concrete input values to 2nd round non-linear layer that produce desired values
        
        # -----------------------------------------------------------------------------------------
        # Find input - output difference for first S- and T-Boxes,
        # such that linear equation system sys1_Do1 becomes square and can be solved via elimination
        # -----------------------------------------------------------------------------------------
        
        # Pick random values for sys1_Do1_nfreevars free variables in Dx
        # Since the valid difference transitions for the S-Box have a lower probability than the T-Box, 
        # it is optimal to have as few S-Boxes as possible to check among the s boxes to check at 
        # the last step.
        
        num_sboxes_guessed = 0
        num_tboxes_guessed = 0
        
        # Make sys2_Di3_sub square by using solutions from sys1_Do1_sub
        if sys1_Do1_nfreevars > 0:
            
            if s1 + t1 < sys1_Do1_nfreevars:
                print("Not solvable")
                return
            
            for i in range(tip.m):
                # First, guess active S-Box differences
                if i in range(0,s1) and s1 != num_sboxes_guessed:
                    # Find valid IO difference pair for S-Box
                    din_S = None
                    while din_S is None:
                        din_S, dout_S = din_to_dout_S(tip, tip.random_element(), u if i == 0 else 8)
                    diffs[P(f'Dx{i}')] = din_S
                    diffs[P(f'Dy{i}')] = dout_S

                    # Find (list of) valid inputs to S-Box that produces IO difference pair
                    ins_S = din_dout_to_in_S(tip, din_S, dout_S)
                    vals[f'x{i}'] = [tip.R_inv * in_S for in_S in ins_S]

                    num_sboxes_guessed += 1
                    sys1_Do1_nfreevars -= 1
                    
                # Second, guess active T-Box differences
                if i in range(tip.s,tip.s+t1) and t1 != num_tboxes_guessed:
                    # Find valid IO difference pair for T-Box
                    din_T, dout_T = din_to_dout_T(tip, tip.random_element())
                    diffs[P(f'Dx{i}')] = din_T
                    diffs[P(f'Dy{i}')] = dout_T

                    # Find (list of) valid inputs to T-Box that produces IO difference pair
                    vals[f'x{i}'] = din_dout_to_in_T(tip, din_T, dout_T)
                    
                    num_tboxes_guessed += 1
                    sys1_Do1_nfreevars -= 1
                
                # Guessed all free variables for sys1_Do1
                if sys1_Do1_nfreevars == 0:
                    break
                    
        assert(sys1_Do1_nfreevars == 0)  
        
        
        if debug > 1:
            print(f"Guessed {num_sboxes_guessed} input difference(s) to {s1} active S-Boxes")
            print(f"Guessed {num_tboxes_guessed} input difference(s) to {t1} active T-Boxes")
            print(diffs)
            print(vals)
        
        # -----------------------------------------------------------------------------------------
        # Substitute into sys1_Do1 (square) and sys2_Di3 (square or overdefined)
        # -----------------------------------------------------------------------------------------
        
        # Substitute S- and T-Box input difference guesses into sys1_Do1
        sys1_Do1_sub = Sequence(sys1_Do1).subs(diffs)
        sys1_Do1_sub_nvars = Sequence(sys1_Do1_sub).nvariables()
        sys1_Do1_sub_nfreevars = sys1_Do1_sub_nvars - sys1_Do1_neqs
        assert(sys1_Do1_sub_nvars == (s1 - num_sboxes_guessed) + (t1 - num_tboxes_guessed))
        
        # Ensure that backward equation system is square.
        assert(sys1_Do1_sub_nfreevars == 0)
        #print(sys1_Do1_sub)
        
        # Substitute S- and T-Box output difference guesses into sys2_Di3
        sys2_Di3_sub = Sequence(sys2_Di3).subs(diffs)
        sys2_Di3_sub_nvars = Sequence(sys2_Di3_sub).nvariables()
        sys2_Di3_sub_nfreevars = sys2_Di3_sub_nvars - sys2_Di3_neqs
        
        # Ensure that forward equation system remains solveable. Since for Tip4 and Tip5, we have c >= s, 
        # this should always be true for our target instances.
        assert(sys2_Di3_sub_nfreevars >= 0)
        #print(eqs_Da3_sub)

        if debug > 1:
            print(f"Substitute possible (din,dout) for {num_sboxes_guessed + num_tboxes_guessed} S- and T-Boxes")
            print(f"Do1: {sys1_Do1_neqs} equations of degree {1} in {sys1_Do1_sub_nvars} variables", flush=True)
            print(f"Di3: {sys2_Di3_neqs} equations of degree {1} in {sys2_Di3_sub_nvars} variables", flush=True)
        
        
        # -----------------------------------------------------------------------------------------
        # Calculate solutions from linear equation system for capacity (sys1_Do1_sub)
        # -----------------------------------------------------------------------------------------
        
        sol_sys1_Do1_sub, _ = solve_linear_equation_system(sys1_Do1_sub)
        if len(sol_sys1_Do1_sub) == 0: 
            continue # No solution found
        diffs.update(sol_sys1_Do1_sub)
        
        # Assumption: s1 S-Boxes and t1 T-boxes are all active
        if not all(diffs.values()):
            continue
        
        # -----------------------------------------------------------------------------------------
        # Find output differences for calculate input-differences to active S- and T-Boxes,
        # such that linear equation system sys2_Di3_sub becomes square and can be solved via elimination
        # -----------------------------------------------------------------------------------------
        
        num_sboxes_fixed = num_sboxes_guessed
        num_tboxes_fixed = num_tboxes_guessed
        
        # Make sys2_Di3_sub square by using solutions from sys1_Do1_sub
        if sys2_Di3_sub_nfreevars > 0:
            
            if sys1_Do1_sub_nvars < sys2_Di3_sub_nfreevars:
                print("Not solvable")
                return
            
            for i in range(tip.m):
                # Calculated S-box difference from linear equation system sys1_Do1_sub
                if i in range(num_sboxes_guessed,s1) and s1 != num_sboxes_fixed:
                    assert(P(f'Dx{i}') in diffs and P(f'Dy{i}') not in diffs)
                    assert(f'x{i}' not in vals)
                    
                    # Find valid output difference for calculated S-Box input difference
                    # TODO: what if there is none? Is this possible?
                    din_S, dout_S = din_to_dout_S(tip, diffs[P(f'Dx{i}')], u=8)
                    diffs[P(f'Dy{i}')] = dout_S
                    
                    # Find (list of) valid inputs to S-Box that produces IO difference pair
                    ins_S = din_dout_to_in_S(tip, din_S, dout_S)
                    vals[f'x{i}'] = [tip.R_inv * in_S for in_S in ins_S]
                    
                    num_sboxes_fixed += 1
                    sys2_Di3_sub_nfreevars -= 1
                
                if i in range(tip.s+num_tboxes_guessed,tip.s+t1) and t1 != num_tboxes_fixed:
                    assert(P(f'Dx{i}') in diffs and P(f'Dy{i}') not in diffs)
                    assert(f'x{i}' not in vals)
                    
                     # Find valid IO difference pair for T-Box
                    din_T, dout_T = din_to_dout_T(tip, diffs[P(f'Dx{i}')])
                    diffs[P(f'Dy{i}')] = dout_T

                    # Find (list of) valid inputs to T-Box that produces IO difference pair
                    vals[f'x{i}'] = din_dout_to_in_T(tip, din_T, dout_T)
                    
                    num_tboxes_fixed += 1
                    sys2_Di3_sub_nfreevars -= 1
                
                # Guessed all free variables for sys2_Di3_sub
                if sys2_Di3_sub_nfreevars == 0:
                    break
                    
        assert(sys2_Di3_sub_nfreevars == 0)   

        if debug > 1:
            print(f"Fixed {num_sboxes_fixed - num_sboxes_guessed} output difference(s) from max {s1 - num_sboxes_guessed} calculated input difference(s) to {s1} active S-Boxes")        
            print(f"Fixed {num_tboxes_fixed - num_tboxes_guessed} output difference(s) from max {t1 - num_tboxes_guessed} calculated input difference(s) to {t1} active T-Boxes")        
        
        # Update sys2_Di3_sub
        sys2_Di3_sub = Sequence(sys2_Di3_sub).subs(diffs)
        sys2_Di3_sub_nvars = Sequence(sys2_Di3_sub).nvariables()
        sys2_Di3_sub_nfreevars = sys2_Di3_sub_nvars - sys2_Di3_neqs

        if debug > 1:
            print(f"Di3: {sys2_Di3_neqs} equations of degree {1} in {sys2_Di3_sub_nvars} variables", flush=True)
        
        assert(sys2_Di3_sub_nfreevars == 0) # Solveable   
        
        # -----------------------------------------------------------------------------------------
        # Calculate solutions from linear equation system for round 3 zero-difference (sys2_Di3_sub)
        # -----------------------------------------------------------------------------------------
        
        sol_sys2_Di3_sub, sys2_Di3_sub = solve_linear_equation_system(sys2_Di3_sub)
        
        if len(sol_sys2_Di3_sub) == 0: 
            continue # No solution found
        diffs.update(sol_sys2_Di3_sub)
        
        # Assumption: s1 S-Boxes and t1 T-boxes are all active
        if not all(diffs.values()):
            continue
        
        # -----------------------------------------------------------------------------------------
        # Check if difference transitions (which have not yet been checked) are valid and 
        # calculate possible inputs
        # -----------------------------------------------------------------------------------------
        
        sc = s1 - num_sboxes_fixed
        tc = t1 - num_tboxes_fixed
        
        if debug > 1:
            print(f"Check validity of difference transitions: sc = {sc}, tc = {tc}.")
            print(diffs)
            
        # Validity of S-box transitions
        if any([valid_din_dout_S(tip, diffs[P(f'Dx{i}')], diffs[P(f'Dy{i}')]) == False for i in range(num_sboxes_fixed, s1)]):
            continue
        
        # Concrete input values for S-Box transitions satisfying IO constraints
        for i in range(num_sboxes_fixed,tip.s):
            # For non-zero difference, get inputs satisfying (din,dout)
            if i < s1:
                ins_S = din_dout_to_in_S(tip, diffs[P(f'Dx{i}')], diffs[P(f'Dy{i}')])
                vals[f'x{i}'] = [tip.R_inv * in_S for in_S in ins_S]
            # For zero-difference, use random value
            elif fixInactiveS:
                vals[f'x{i}'] = [tip.random_element()]
        
        # Validity of T-Box transitions and concrete inputs
        all_tbox_trans_valid = True
        for i in range(tip.s+num_tboxes_fixed, tip.s+t1):
            # Find (list of) valid inputs to T-Box that produces IO difference pair
            vals[f'x{i}'] = din_dout_to_in_T(tip, diffs[P(f'Dx{i}')], diffs[P(f'Dy{i}')])
            if len(vals[f'x{i}']) == 0:
                all_tbox_trans_valid = False
            
        if not all_tbox_trans_valid:
            continue
        
        # -----------------------------------------------------------------------------------------
        # Double-check that linear difference constraints are indeed fulfilled
        # -----------------------------------------------------------------------------------------
        
        Di2_sub = Di2.subs(diffs)
        Do2_sub = Do2.subs(diffs)
        
        # Backward: Conditions on initial capacity part in first round
        Do1_sub = tip.multiply_mds_matrix(Di2_sub, inv=True)
        assert(all([Do1_sub[i] == 0 for i in range(tip.r,tip.m)]))
        
        # Forward: No activate S-boxes in third round
        Di3_sub = tip.multiply_mds_matrix(Do2_sub, inv=False)
        assert(all([Di3_sub[i] == 0 for i in range(tip.s)]))
        
        # -----------------------------------------------------------------------------------------
        # Double-check that concrete values lead to corresponding differences
        # -----------------------------------------------------------------------------------------
        
        for i in range(0, s1):
            # S(x+dx) - S(x) = dy
            assert(all([tip.S(x + Di2_sub[i]) - tip.S(x) == Do2_sub[i] for x in vals[f'x{i}']]))
        
        for i in range(tip.s, tip.s+t1):
            # T(x+dx) - T(x) = dy
            assert(all([tip.T(x + Di2_sub[i]) - tip.T(x) == Do2_sub[i] for x in vals[f'x{i}']]))
        
        
        found_solution = True
        if debug > 0:
            print("Number of tries: ", numtries)
            print(f"Guessed {num_sboxes_guessed} input difference(s) to {s1} active S-Boxes")
            print(f"Guessed {num_tboxes_guessed} input difference(s) to {t1} active T-Boxes")
            print(f"Fixed {num_sboxes_fixed - num_sboxes_guessed} output difference(s) from max {s1 - num_sboxes_guessed} calculated input difference(s) to {s1} active S-Boxes")        
            print(f"Fixed {num_tboxes_fixed - num_tboxes_guessed} output difference(s) from max {t1 - num_tboxes_guessed} calculated input difference(s) to {t1} active T-Boxes")  
    
    # vals     dictionary with possible concrete values for i2 (s1+t1 variables)
    # Di2_sub  vector with concrete input differences to 2nd round nonlinear layer
    # Do2_sub  vector with concrete output differences to 2nd round nonlinear layer
    return vals, Di2_sub, Do2_sub, numtries

In [11]:
two_stage_ca_stage1(tip5, s1=1, t1=6, u=1, debug=1, fixInactiveS=False);

Di2 =  (18446744065119617025*Dx0, 0, 0, 0, Dx4, Dx5, Dx6, Dx7, Dx8, Dx9, 0, 0, 0, 0, 0, 0)
Do2 =  (18446744065119617025*Dy0, 0, 0, 0, Dy4, Dy5, Dy6, Dy7, Dy8, Dy9, 0, 0, 0, 0, 0, 0)
Sys1 (Do1): 6 equations of degree 1 in 7 variables
Sys2 (Di3): 4 equations of degree 1 in 7 variables
Number of tries:  22
Guessed 1 input difference(s) to 1 active S-Boxes
Guessed 0 input difference(s) to 6 active T-Boxes
Fixed 0 output difference(s) from max 0 calculated input difference(s) to 1 active S-Boxes
Fixed 2 output difference(s) from max 6 calculated input difference(s) to 6 active T-Boxes


In [12]:
#two_stage_ca_stage1(tip4, s1=4, t1=1, u=1, debug=1, fixInactiveS=True)

In [13]:
two_stage_ca_stage1(tip4p, s1=1, t1=4, u=1, debug=1, fixInactiveS=False);

Di2 =  (18446744065119617025*Dx0, 0, 0, 0, Dx4, Dx5, Dx6, Dx7, 0, 0, 0, 0)
Do2 =  (18446744065119617025*Dy0, 0, 0, 0, Dy4, Dy5, Dy6, Dy7, 0, 0, 0, 0)
Sys1 (Do1): 4 equations of degree 1 in 5 variables
Sys2 (Di3): 4 equations of degree 1 in 5 variables
Number of tries:  4
Guessed 1 input difference(s) to 1 active S-Boxes
Guessed 0 input difference(s) to 4 active T-Boxes
Fixed 0 output difference(s) from max 0 calculated input difference(s) to 1 active S-Boxes
Fixed 0 output difference(s) from max 4 calculated input difference(s) to 4 active T-Boxes


### Section 5.2: Stage 2 - Find collision

In [14]:
def two_stage_ca_stage2(tip, i2f, t1, Di2, Do2, debug=False):
    '''
    tip   Tip5 like instance
    i2f   Dictionary of s+t1 fixed inputs to s S-Boxes and first t1 T-Boxes
    t1    Number of active T-Boxes (first t1 of t = tip.m-tip.s T-Boxes are active)
    Di2   Vector with concrete input differences to 2nd round nonlinear layer
    Do2   Vector with concrete output differences to 2nd round nonlinear layer
    '''        
    print("Fixed values: \n",i2f)
    
    # Free variables
    nfreevars = (tip.m - (tip.s + t1)) - (tip.c + tip.d)
    
    # ----------------------------------------
    # Variables
    # ----------------------------------------
    
    varnames = [f'x{i}' for i in range(tip.s+t1+nfreevars,tip.m)]
    P = PolynomialRing(tip.field, varnames, order='degrevlex')
    
    # ----------------------------------------
    # Input vectors to second round non-linear layer
    # ----------------------------------------
    
    i2 = vector(P, tip.m)
    for i in range(0,tip.m):
        x = f'x{i}'
        if i < tip.s+t1:
            i2[i] = i2f[x]
        else:
            if nfreevars > 0:
                print(f"Randomly fix value for {x}.")
                i2f[x] = tip.field.random_element()
                i2[i] = i2f[x]
                nfreevars -= 1
            else:
                i2[i] = P(x)
    
    Di2 = Di2.change_ring(P)
    I2 = i2 + Di2
    
    # ----------------------------------------
    # Equations
    # ----------------------------------------
    
    # Backward: Conditions on initial capacity part in first round
    assert(tip.r >= tip.s)
    o1 = tip.linear_layer(i2, r=0, inv=True)
    eqs_o1 = [(o1[i] - tip.S(tip.c_value)) if (i < tip.s) else (o1[i] - tip.T(tip.c_value)) for i in range(tip.r,tip.m)]
    assert(all([f.degree() == 1 for f in eqs_o1]))
    
    o1_neqs = len(eqs_o1)
    o1_nvars = Sequence(eqs_o1).nvariables()
    
    print(f"o1:  {o1_neqs} equations of degree {1} in {o1_nvars} variables", flush=True)
    assert(o1_nvars >= o1_neqs)
    
    # Forward: Collision in hash
    o2 = tip.nonlinear_layer(i2)
    O2 = tip.nonlinear_layer(I2)
    
    Do2 = Do2.change_ring(P)
    assert(O2 - o2 == Do2)
    
    i3 = tip.linear_layer(o2, r=1, inv=False)
    I3 = tip.linear_layer(O2, r=1, inv=False)
    
    i3[:tip.s] = [0]*tip.s  # Ignore first s entries for nonlinear layer
    I3[:tip.s] = [0]*tip.s  # Ignore first s entries for nonlinear layer
    
    o3 = tip.nonlinear_layer(i3)  # First s 0 entries: 0 -> 0
    O3 = tip.nonlinear_layer(I3)  # First s 0 entries: 0 -> 0
    assert(all([f.degree() == tip.alpha**2 for f in o3[tip.s:]]))
    assert(all([f.degree() == tip.alpha**2 for f in O3[tip.s:]]))
    assert(all([fo != fO for fo,fO in zip(o3[tip.s:],O3[tip.s:])]))
    
    Do3 = O3 - o3
    assert(all([f == 0 for f in Do3[:tip.s]]))
    
    #assert(all([f.degree() == tip.alpha**2 - 1 for f in Do3[tip.s:]])) # Because of fixed differences
    assert(all([f.degree() == tip.alpha * (tip.alpha - 1) for f in Do3[tip.s:]])) # Because of fixed differences

    Do = tip.multiply_mds_matrix(Do3) # output difference (without RC addition, because of difference)
    eqs_Do = [Do[i] for i in range(tip.d)]
    
    Do_neqs = len(eqs_Do)
    Do_nvars = Sequence(eqs_Do).nvariables()
    
    print(f"Do:  {Do_neqs} equations of degree {tip.alpha * (tip.alpha - 1)} in {Do_nvars} variables", flush=True)
    assert(Do_nvars >= Do_neqs)
    
    return eqs_o1 + eqs_Do, i2f

### Section 5.3: Applications to the Collision Attack on 3-Round Tip4

In [15]:
def two_stage_ca(tip, s1, t1, u, debug=False, writeFile=False, nearCollision=False, ge=False):
    
    # Perform step 1
    print('-'*80 + "\nTwo stage collision attack: Stage 1\n" + '-'*80)
    i2_vals, Di2, Do2, _ = two_stage_ca_stage1(tip, s1, t1, u, debug, fixInactiveS=True)
    assert(len(i2_vals) == tip.s + t1)
    
    # Fixed values from step 1
    i2f = {}
    for v in i2_vals:
        i2f[v] = i2_vals[v][0] # TODO maybe random, or all?
    
    # Generate equation system for step 2
    print('-'*80 + "\nTwo stage collision attack: Equations for Stage 2\n" + '-'*80)
    eqs, i2f = two_stage_ca_stage2(tip, i2f, t1, Di2, Do2, debug)
    
    if ge:
        # Perform Gaussian Elimination
        print('-'*80 + "\nPerform Gaussian Elimination\n" + '-'*80)
        eqs, sub_lin = gaussian_elimination(eqs)
        #print(sub_lin)
        fixed_elems = [i2f,Di2,Do2,sub_lin]
        mdir = "magma/GE"
    else:
        fixed_elems = [i2f,Di2,Do2]
        mdir = "magma"
    
    print_stats(eqs)

    if writeFile:
        if nearCollision:
            gen_magma_file_tip(tip, eqs, fixed_elems=fixed_elems, attacktype=f"two_stage_nca_{tip.name.lower()}_d{tip.d}_s1_{s1}_t1_{t1}_u_{u}", nonzero=None, mdir=mdir, replace=False)
        else:
            gen_magma_file_tip(tip, eqs, fixed_elems=fixed_elems, attacktype=f"two_stage_ca_{tip.name.lower()}_s1_{s1}_t1_{t1}_u_{u}", nonzero=None, mdir=mdir, replace=False)

In [16]:
def two_stage_ca_find_params(tip, square=False):
    s = tip.s
    t = tip.m - tip.s
    
    candidates = []
    
    for s1 in range(s + 1):
        for t1 in range(t + 1):
            s0 = s - s1
            t0 = t - t1
            
            # Stage 1: s1 + t1 > max(c,s)
            # Stage 2: t0 >= c + d
            if (s1 + t1 > max(tip.c,tip.s)) and (t0 >= tip.c + tip.d):
                if square:
                    ne = tip.c + tip.d + t0
                    nv = 2*t0
                    if ne == nv:
                        candidates.append((s1,t1))
                else:
                    candidates.append((s1,t1))

    return candidates

In [17]:
for tip in [tip5, tip4, tip4p]:
    params = two_stage_ca_find_params(tip, square=False)
    print(tip.name)
    print(params)

Tip5
[]
TIP4
[(1, 4), (2, 3), (2, 4), (3, 2), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4)]
TIP4p
[]


#### Tip4 CA

In [18]:
two_stage_ca(tip4, s1=1, t1=4, u=1, debug=1, writeFile=False, ge=False) # without a priori performing Gaussian elimination

--------------------------------------------------------------------------------
Two stage collision attack: Stage 1
--------------------------------------------------------------------------------
Di2 =  (18446744065119617025*Dx0, 0, 0, 0, Dx4, Dx5, Dx6, Dx7, 0, 0, 0, 0, 0, 0, 0, 0)
Do2 =  (18446744065119617025*Dy0, 0, 0, 0, Dy4, Dy5, Dy6, Dy7, 0, 0, 0, 0, 0, 0, 0, 0)
Sys1 (Do1): 4 equations of degree 1 in 5 variables
Sys2 (Di3): 4 equations of degree 1 in 5 variables
Number of tries:  28
Guessed 1 input difference(s) to 1 active S-Boxes
Guessed 0 input difference(s) to 4 active T-Boxes
Fixed 0 output difference(s) from max 0 calculated input difference(s) to 1 active S-Boxes
Fixed 0 output difference(s) from max 4 calculated input difference(s) to 4 active T-Boxes
--------------------------------------------------------------------------------
Two stage collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fixed value

#### Tip4 Near-CA (d=2)

In [19]:
tip4_d2 = Tip5(name="TIP4", rate=12, capacity=4, digest_length=2)

In [20]:
two_stage_ca(tip4_d2, s1=1, t1=4, u=1, debug=1, writeFile=False, nearCollision=True, ge=True)

--------------------------------------------------------------------------------
Two stage collision attack: Stage 1
--------------------------------------------------------------------------------
Di2 =  (18446744065119617025*Dx0, 0, 0, 0, Dx4, Dx5, Dx6, Dx7, 0, 0, 0, 0, 0, 0, 0, 0)
Do2 =  (18446744065119617025*Dy0, 0, 0, 0, Dy4, Dy5, Dy6, Dy7, 0, 0, 0, 0, 0, 0, 0, 0)
Sys1 (Do1): 4 equations of degree 1 in 5 variables
Sys2 (Di3): 4 equations of degree 1 in 5 variables
Number of tries:  164
Guessed 1 input difference(s) to 1 active S-Boxes
Guessed 0 input difference(s) to 4 active T-Boxes
Fixed 0 output difference(s) from max 0 calculated input difference(s) to 1 active S-Boxes
Fixed 0 output difference(s) from max 4 calculated input difference(s) to 4 active T-Boxes
--------------------------------------------------------------------------------
Two stage collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fixed valu

### Section 5.4: Straightforward Applications to 3-Round SFS Collision Attacks

In [21]:
def two_stage_sfs_ca_stage2(tip, i2f, s1, t1, Di2, Do2, debug=False):
    '''
    tip   Tip5 like instance
    i2f   Dictionary of s+t1 fixed inputs to s S-Boxes and first t1 T-Boxes
    s1    Number of active S-Boxes (first s1 of tip.s S-Boxes are active)
    t1    Number of active T-Boxes (first t1 of t = tip.m-tip.s T-Boxes are active)
    Di2   Vector with concrete input differences to 2nd round nonlinear layer
    Do2   Vector with concrete output differences to 2nd round nonlinear layer
    '''   
    
    if debug:
        print("Fixed values: \n",i2f)      
    
    # ----------------------------------------
    # Variables
    # ----------------------------------------
    
    t = tip.m - tip.s  # Number of T-Boxes
    s0 = tip.s - s1    # Number of inactive S-Boxes
    t0 = t - t1        # Number of inactive T-Boxes
    
    # Randomly fix s0+t0-d values. Assumption: Fix as many inputs to inactive S-Boxes, then T-Boxes
    assert(s0 + t0 > tip.d)
    
    s0f = min(s0, tip.d, s0 + t0 - tip.d)
    t0f = (s0 + t0 - tip.d) - s0f
    
    print(f"Fix {s0f} outputs to {s0} = {tip.s} - {s1} inactive S-Boxes.")
    print(f"Fix {t0f} outputs to {t0} = {tip.m - tip.s} - {t1} inactive T-Boxes.")
    
    if t0f < 0:
        print("Not possible to fix enough inputs/outputs to non-active S- and T-Boxes")
        return
    print(f"Remaining variables: {(s0 - s0f) + (t0 - t0f)}")
    
    # Variables only for remaining parts
    varnames = [f'y{i}' for i in range(s1 + s0f, tip.s)] + [f'y{i}' for i in range(tip.s + t1 + t0f, tip.m)]
    P = PolynomialRing(tip.field, varnames, order='degrevlex')
    
    Di2 = Di2.change_ring(tip.field)
    o2 = vector(P, tip.m) # Outputs to second round non-linear layer
    O2 = vector(P, tip.m) # Outputs to second round non-linear layer

    for i in range(tip.m):
        x,y = f'x{i}', f'y{i}'
        if i < s1:
            # Values calculated in step 1
            o2[i] = tip.S(i2f[x])
            O2[i] = tip.S(i2f[x] + Di2[i])
            assert(Di2[i] != 0)
        elif i < s1 + s0f:
            # Random fixes, 0 difference
            x_val = tip.random_element()
            o2[i] = tip.S(x_val)
            O2[i] = o2[i]
            i2f[x] = x_val
        elif i < tip.s:
            # Variables here with 0 difference
            o2[i] = P(y)
            O2[i] = o2[i]
        elif i < tip.s + t1:
            # Values calculated in step 1
            o2[i] = tip.T(i2f[x])
            O2[i] = tip.T(i2f[x] + Di2[i])
            assert(Di2[i] != 0)
        elif i < tip.s + t1 + t0f:
            # Random fixes, 0 difference
            x_val = tip.random_element()
            o2[i] = tip.T(x_val)
            O2[i] = o2[i]
            i2f[x] = x_val
        else:
            # Variables here with 0 difference
            o2[i] = P(y)
            O2[i] = o2[i]
    
    if debug:
        print(o2)
        print(O2)
    
    Do2 = Do2.change_ring(P)
    assert(O2 - o2 == Do2)
    
    if debug:
        print("Fixed values: \n",i2f)
    
    # ----------------------------------------
    # Equations
    # ----------------------------------------
    
    # Forward: Collision in hash
    i3 = tip.linear_layer(o2, r=1, inv=False)
    I3 = tip.linear_layer(O2, r=1, inv=False)
    
    i3[:tip.s] = [0]*tip.s  # Ignore first s entries for nonlinear layer
    I3[:tip.s] = [0]*tip.s  # Ignore first s entries for nonlinear layer
    
    o3 = tip.nonlinear_layer(i3)  # First s 0 entries: 0 -> 0
    O3 = tip.nonlinear_layer(I3)  # First s 0 entries: 0 -> 0
    assert(all([f.degree() == tip.alpha for f in o3[tip.s:]]))
    assert(all([f.degree() == tip.alpha for f in O3[tip.s:]]))
    assert(all([fb != fB for fb,fB in zip(o3[tip.s:],O3[tip.s:])]))
    Do3 = O3 - o3
    assert(all([f == 0 for f in Do3[:tip.s]]))
    assert(all([f.degree() == tip.alpha - 1 for f in Do3[tip.s:]])) # Because of zero-difference in unknown parts
    
    Do = tip.multiply_mds_matrix(Do3) # output difference (without RC addition, because of difference)
    eqs_Do = [Do[i] for i in range(tip.d)]
    
    Do_neqs = len(eqs_Do)
    Do_nvars = Sequence(eqs_Do).nvariables()
    
    print(f"Do:  {Do_neqs} equations of degree {tip.alpha - 1} in {Do_nvars} variables", flush=True)
    assert(Do_nvars == Do_neqs)
    
    return eqs_Do, i2f

In [22]:
def two_stage_sfs_ca(tip, s1, t1, u, debug=False, writeFile=False):
    '''
    tip   Tip5 like instance
    s1    Number of active S-Boxes (first s1 of tip.s S-Boxes are active)
    t1    Number of active T-Boxes (first t1 of t = tip.m-tip.s T-Boxes are active)
    u     Number of active chunks in s1-th S-Box
    '''   
        
    # Perform step 1
    print('='*80 + "\nTwo stage SFS collision attack: Stage 1\n" + '-'*80)
    i2_vals, Di2, Do2, num_tries = two_stage_ca_stage1(tip, s1, t1, u, debug, fixInactiveS=False)
    assert(len(i2_vals) == s1 + t1)
    print(f"Number of tries: {num_tries}")
    
    # Fixed values from step 1
    i2f = {}
    for v in i2_vals:
        i2f[v] = i2_vals[v][0] # TODO maybe random, or all?
    
    # Generate equation system for step 2
    print('-'*80 + "\nTwo stage SFS collision attack: Equations for Stage 2\n" + '-'*80)
    eqs, i2f = two_stage_sfs_ca_stage2(tip, i2f, s1, t1, Di2, Do2, debug)
    
    fixed_elems = [i2f,Di2,Do2]
    mdir = "magma"
    
    print_stats(eqs)
    
    if writeFile:
        gen_magma_file_tip(tip, eqs, fixed_elems=fixed_elems, attacktype=f"two_stage_sfs_ca_{tip.name.lower()}_s1_{s1}_t1_{t1}_u_{u}", nonzero=None, mdir=mdir, replace=False)

In [23]:
def two_stage_sfs_ca_find_params(tip):
    s = tip.s
    t = tip.m - tip.s
    
    candidates = []
    
    for s1 in range(s + 1):
        for t1 in range(t + 1):
            s0 = s - s1
            t0 = t - t1
            
            # Stage 1: s1 + t1 > max(c,s)
            # Stage 2: s0 + t0 >= d
            
            if (s1 + t1 > max(tip.c,tip.s)) and (s0 + t0 >= tip.d):
                candidates.append((s1,t1))

    return candidates

In [24]:
for tip in [tip5, tip4, tip4p]:
    params = two_stage_sfs_ca_find_params(tip)
    print(tip.name)
    print(params)

Tip5
[(0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7)]
TIP4
[(0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8)]
TIP4p
[(0, 5), (0, 6), (0, 7), (0, 8), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (3, 2), (3, 3), (3, 4), (3, 5), (4, 1), (4, 2), (4, 3), (4, 4)]


#### Tip5

In [25]:
two_stage_sfs_ca(tip5, s1=0, t1=7, u=1, debug=0, writeFile=False)

Two stage SFS collision attack: Stage 1
--------------------------------------------------------------------------------
Number of tries: 33
--------------------------------------------------------------------------------
Two stage SFS collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fix 4 outputs to 4 = 4 - 0 inactive S-Boxes.
Fix 0 outputs to 5 = 12 - 7 inactive T-Boxes.
Remaining variables: 5
Do:  5 equations of degree 6 in 5 variables
5 equations in 5 variables
--------------------------------------------------------------------------------
Macaulay bound:  26
GB complexity:   17.4w + 0.0 (41.2)
--------------------------------------------------------------------------------
Bézout bound:    2^5 * 3^5
FGLM complexity: 13.0w + 2.4 (41.4)


In [26]:
two_stage_sfs_ca(tip5, s1=1, t1=6, u=1, debug=0, writeFile=False)

Two stage SFS collision attack: Stage 1
--------------------------------------------------------------------------------
Number of tries: 69
--------------------------------------------------------------------------------
Two stage SFS collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fix 3 outputs to 3 = 4 - 1 inactive S-Boxes.
Fix 1 outputs to 6 = 12 - 6 inactive T-Boxes.
Remaining variables: 5
Do:  5 equations of degree 6 in 5 variables
5 equations in 5 variables
--------------------------------------------------------------------------------
Macaulay bound:  26
GB complexity:   17.4w + 0.0 (41.2)
--------------------------------------------------------------------------------
Bézout bound:    2^5 * 3^5
FGLM complexity: 13.0w + 2.4 (41.4)


#### Tip4

In [27]:
two_stage_sfs_ca(tip4, s1=0, t1=5, u=1, debug=0, writeFile=False)

Two stage SFS collision attack: Stage 1
--------------------------------------------------------------------------------
Number of tries: 17
--------------------------------------------------------------------------------
Two stage SFS collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fix 4 outputs to 4 = 4 - 0 inactive S-Boxes.
Fix 3 outputs to 7 = 12 - 5 inactive T-Boxes.
Remaining variables: 4
Do:  4 equations of degree 6 in 4 variables
4 equations in 4 variables
--------------------------------------------------------------------------------
Macaulay bound:  21
GB complexity:   13.7w + 0.0 (32.5)
--------------------------------------------------------------------------------
Bézout bound:    2^4 * 3^4
FGLM complexity: 10.4w + 2.0 (33.2)


In [28]:
two_stage_sfs_ca(tip4, s1=1, t1=4, u=1, debug=0, writeFile=False)

Two stage SFS collision attack: Stage 1
--------------------------------------------------------------------------------
Number of tries: 75
--------------------------------------------------------------------------------
Two stage SFS collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fix 3 outputs to 3 = 4 - 1 inactive S-Boxes.
Fix 4 outputs to 8 = 12 - 4 inactive T-Boxes.
Remaining variables: 4
Do:  4 equations of degree 6 in 4 variables
4 equations in 4 variables
--------------------------------------------------------------------------------
Macaulay bound:  21
GB complexity:   13.7w + 0.0 (32.5)
--------------------------------------------------------------------------------
Bézout bound:    2^4 * 3^4
FGLM complexity: 10.4w + 2.0 (33.2)


#### Tip4'

In [29]:
two_stage_sfs_ca(tip4p, s1=0, t1=5, u=1, debug=0, writeFile=False)

Two stage SFS collision attack: Stage 1
--------------------------------------------------------------------------------
Number of tries: 86
--------------------------------------------------------------------------------
Two stage SFS collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fix 3 outputs to 4 = 4 - 0 inactive S-Boxes.
Fix 0 outputs to 3 = 8 - 5 inactive T-Boxes.
Remaining variables: 4
Do:  4 equations of degree 6 in 4 variables
4 equations in 4 variables
--------------------------------------------------------------------------------
Macaulay bound:  21
GB complexity:   13.7w + 0.0 (32.5)
--------------------------------------------------------------------------------
Bézout bound:    2^4 * 3^4
FGLM complexity: 10.4w + 2.0 (33.2)


In [30]:
two_stage_sfs_ca(tip4p, s1=1, t1=4, u=1, debug=0, writeFile=False)

Two stage SFS collision attack: Stage 1
--------------------------------------------------------------------------------
Number of tries: 29
--------------------------------------------------------------------------------
Two stage SFS collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fix 3 outputs to 3 = 4 - 1 inactive S-Boxes.
Fix 0 outputs to 4 = 8 - 4 inactive T-Boxes.
Remaining variables: 4
Do:  4 equations of degree 6 in 4 variables
4 equations in 4 variables
--------------------------------------------------------------------------------
Macaulay bound:  21
GB complexity:   13.7w + 0.0 (32.5)
--------------------------------------------------------------------------------
Bézout bound:    2^4 * 3^4
FGLM complexity: 10.4w + 2.0 (33.2)


## Section 7: Application to  Round-Reduced Monolith

### Section 7.2: Application to 2-Round Collision Attacks

#### Stage 1: Find non-zero differences and valid values

In [31]:
# This is done in C++, read in from file
def mono_two_stage_ca_stage1(mono, filename):
    from ast import literal_eval
    with open(filename) as f:
        # Di1 
        Di1 = literal_eval(f.readline()[6:-1])
        Do1 = literal_eval(f.readline()[6:-1])
        i1f = literal_eval(f.readline()[6:-1])
    
    Di1 = vector(mono.F, Di1)
    Do1 = vector(mono.F, Do1)
    i1f = {k: mono.i2f(v) for k,v in i1f.items()}
    return i1f, Di1, Do1

In [32]:
mono_two_stage_ca_stage1(Mono31, "monolith-31-stage1.txt")

({'x0': 1769176292,
  'x1': 474569811,
  'x2': 2112689211,
  'x3': 1266486537,
  'x4': 1481086973,
  'x5': 1130812394,
  'x6': 1590835993,
  'x7': 1234430009},
 (1630247904, 2107149975, 1802005106, 1378426179, 350811560, 815160035, 1958855613, 1335208794, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 645582796),
 (634041782, 915772315, 2038072678, 724312696, 1712880358, 198008050, 1958559545, 1759016090, 2000721589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 645582796))

#### Stage 2: Find (inner) collision on 2-round Monolith

In [33]:
def mono_two_stage_ca_stage2(mono, i1f, Di1, Do1, innerCollison, debug=False):
    '''
    mono  Monolith instance
    i2f   Dictionary of s first fixed inputs to 1st non-linear layer FB
    Di1   Vector with concrete input differences to 1st non-linear layer FB
    Do1   Vector with concrete output differences to 1st non-linear layer FB
    innerCollison    
          If true: target collision in the last c truncated words.
          If false: target collision in the first d truncated words (hash value).
    '''        
    print("Fixed values: \n",i1f)
    
    # ----------------------------------------
    # Variables
    # ----------------------------------------
    
    varnames = [f'x{i}' for i in range(mono.u,mono.t)]
    #P = PolynomialRing(mono.F, varnames, order='degrevlex')
    P = PolynomialRing(mono.F, varnames, order='lex')
    if debug:
        print(P)
    Di1 = Di1.change_ring(P)
    Do1 = Do1.change_ring(P)
    
    # ----------------------------------------
    # Input vectors to first round non-linear layer
    # ----------------------------------------
    
    i1 = vector(P, mono.t)
    for i in range(0,mono.t):
        x = f'x{i}'
        i1[i] = i1f[x] if i < mono.u else P(x)
    I1 = i1 + Di1
    #print(i1)
    
    # ----------------------------------------
    # Verify conditions from stage 1
    # ----------------------------------------
    print("-"*20)
    
    # (1) Zero difference in inner part
    i0 = mono.concrete(i1, inv=True) # invert initial CONCRETE layer
    I0 = mono.concrete(I1, inv=True) 
    Di0 = I0 - i0
    #print(Di0)
    assert(all([Di0[i] == mono.c_value for i in range(mono.t-mono.c,mono.t)]))
    
    # (2) Calculated output difference
    o1 = mono.nonlinear_layer(i1)
    O1 = mono.nonlinear_layer(I1)
    assert(o1[mono.u].degree() == 1 and all([o1[i].degree() == 2 for i in range(mono.u+1,mono.t)]))
    Do1_ = O1 - o1
    if debug:
        print("Calculated output differences 1st non-linear layer match differences retrieved in stage 1 :\n",[Do1[i] == Do1_[i] for i in range(mono.t)])
    assert(Do1 == Do1_)
    
    # (2) Zero-difference for S-Box input at second round
    i2 = mono.linear_layer(o1, r=0)
    I2 = mono.linear_layer(O1, r=0)
    Di2 = I2 - i2
    if debug:
        print(f"First {mono.u} input differences 2nd non-linear layer (should all be 0):\n",Di2[:mono.u])
    assert(all([Di2[i] == 0 for i in range(mono.u)]))
    
    # ----------------------------------------
    # Equations
    # ----------------------------------------
    print("-"*20)
    
    # (1) Backward: Match inner part
    eqs_mip = [i0[i] - mono.c_value for i in range(mono.t-mono.c,mono.t)] # match inner part
    assert(all([f.degree() == 1 for f in eqs_mip]))
    #print(eqs_mip)
    
    eqs_mip_neqs = len(eqs_mip)
    eqs_mip_nvars = Sequence(eqs_mip).nvariables()
    print(f"MIP: {eqs_mip_neqs} equations of degree {1} in {eqs_mip_nvars} variables", flush=True)
    assert(eqs_mip_nvars >= eqs_mip_neqs)
    
    # (2) Forward: Collision (= zero difference) 
    # If innerCollision is true: in inner part (c state words)
    # If innerCollision is false: in hash (d state words)
    
    # Ignore first u entries for nonlinear layer. 
    # The first u+1 outputs of the nonlinear layer will not be correct, but all differences will be correct.
    i2[:mono.u] = [0]*mono.u  
    I2[:mono.u] = [0]*mono.u
    
    o2 = mono.nonlinear_layer(i2)
    O2 = mono.nonlinear_layer(I2)
    Do2 = O2 - o2
    if debug:
        print(f"First {mono.u+1} output differences 2nd non-linear layer (first {mono.u} should be 0, Do2[{mono.u}] should be {Di2[mono.u]}):\n", Do2[:mono.u+1])
    assert(all([Do2[i] == 0 for i in range(mono.u)]))
    assert(Do2[mono.u] == Di2[mono.u])
    if debug:
        print(f"Degrees output differences 2nd non-linear layer:\n", [f.degree() for f in Do2])
    
    # Output differences (without RC addition, because of difference)
    Do = mono.concrete(Do2)
    if debug:
        print(f"Degrees output differences:\n", [f.degree() for f in Do])
    assert(all([f.degree() == 2 for f in Do])) # TODO fukang wrote 3
    
    if innerCollison:
        eqs_col = [Do[i] for i in range(mono.r, mono.t)] # zero difference in inner part
        assert(len(eqs_col) == mono.c)
    else:
        eqs_col = [Do[i] for i in range(mono.d)] # zero hash difference
    eqs_col_neqs = len(eqs_col)
    eqs_col_nvars = Sequence(eqs_col).nvariables()
    
    print(f"ZHD: {eqs_col_neqs} equations of degree {2} in {eqs_col_nvars} variables", flush=True)
    assert(eqs_col_nvars >= eqs_col_neqs)
    
    show_hex=True
    if debug:
        print("Inputs")
        print(f"x[:{mono.u}] = {[hex(mono.f2i(xi)) if show_hex else mono.f2i(xi) for xi in i1[:mono.u]]}")
        print(f"x_[:{mono.u}] = {[hex(mono.f2i(xi)) if show_hex else mono.f2i(xi) for xi in I1[:mono.u]]}")
        print(f"Dx[:{mono.u}] = {[hex(mono.f2i(xi)) if show_hex else mono.f2i(xi) for xi in Di1[:mono.u]]}")

        ob1 = mono.bars(i1)
        Ob1 = mono.bars(I1)
        Dob1 = Ob1-ob1

        print("Only BARS")
        print(f"y[:{mono.u}] = {[hex(mono.f2i(yi)) if show_hex else mono.f2i(yi) for yi in ob1[:mono.u]]}")
        print(f"y_[:{mono.u}] = {[hex(mono.f2i(yi)) if show_hex else mono.f2i(yi) for yi in Ob1[:mono.u]]}")
        print(f"Dy[:{mono.u}] = {[hex(mono.f2i(yi)) if show_hex else mono.f2i(yi) for yi in Dob1[:mono.u]]}")

        print("BARS and Bricks")
        print(f"y[:{mono.u}] = {[hex(mono.f2i(yi)) if show_hex else mono.f2i(yi) for yi in o1[:mono.u]]}")
        print(f"y_[:{mono.u}] = {[hex(mono.f2i(yi)) if show_hex else mono.f2i(yi) for yi in O1[:mono.u]]}")
        print(f"Dy[:{mono.u}] = {[hex(mono.f2i(yi)) if show_hex else mono.f2i(yi) for yi in Do1_[:mono.u]]}")
    
    return eqs_mip + eqs_col

In [34]:
def mono_two_stage_ca(mono, filename, debug=False, writeFile=False, innerCollison=False, ge=False):
    # Perform step 1
    print('='*80 + "\nTwo stage collision attack: Stage 1\n" + '-'*80)
    i1f, Di1, Do1 = mono_two_stage_ca_stage1(mono, filename)
    if debug:
        print("i1f = ", i1f)
        print("Di1 = ", Di1)
        print("Do1 = ", Do1)
    
    # Generate equation system for step 2
    print('-'*80 + "\nTwo stage collision attack: Equations for Stage 2\n" + '-'*80)
    eqs = mono_two_stage_ca_stage2(mono, i1f, Di1, Do1, innerCollison=innerCollison, debug=debug)
    
    if ge:
        # Perform Gaussian Elimination
        print('-'*80 + "\nPerform Gaussian Elimination\n" + '-'*80)
        eqs, sub_lin = gaussian_elimination(eqs)
        #print(sub_lin)
        fixed_elems = [i1f,Di1,Do1,sub_lin]
        mdir = "magma/GE"
    else:
        fixed_elems = [i1f,Di1,Do1]
        mdir = "magma"
    
    print_stats(eqs)
    
    if writeFile:
        gen_magma_file_mono(mono, eqs, fixed_elems=fixed_elems, attacktype=f"two_stage_{'i' if innerCollison else ''}ca_{mono.name.lower()}", mdir=mdir, replace=False) 
    return eqs

In [35]:
mono_two_stage_ca(Mono31, "monolith-31-stage1.txt", innerCollison=True, writeFile=False, ge=False, debug=True);

Two stage collision attack: Stage 1
--------------------------------------------------------------------------------
i1f =  {'x0': 1769176292, 'x1': 474569811, 'x2': 2112689211, 'x3': 1266486537, 'x4': 1481086973, 'x5': 1130812394, 'x6': 1590835993, 'x7': 1234430009}
Di1 =  (1630247904, 2107149975, 1802005106, 1378426179, 350811560, 815160035, 1958855613, 1335208794, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 645582796)
Do1 =  (634041782, 915772315, 2038072678, 724312696, 1712880358, 198008050, 1958559545, 1759016090, 2000721589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 645582796)
--------------------------------------------------------------------------------
Two stage collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fixed values: 
 {'x0': 1769176292, 'x1': 474569811, 'x2': 2112689211, 'x3': 1266486537, 'x4': 1481086973, 'x5': 1130812394, 'x6': 1590835993, 'x7': 1234430009}
Multivariate Polynomial Ring in x8, x

In [36]:
mono_two_stage_ca(Mono31, "monolith-31-stage1.txt", innerCollison=True, writeFile=False, ge=True, debug=False);

Two stage collision attack: Stage 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Two stage collision attack: Equations for Stage 2
--------------------------------------------------------------------------------
Fixed values: 
 {'x0': 1769176292, 'x1': 474569811, 'x2': 2112689211, 'x3': 1266486537, 'x4': 1481086973, 'x5': 1130812394, 'x6': 1590835993, 'x7': 1234430009}
--------------------
--------------------
MIP: 8 equations of degree 1 in 16 variables
ZHD: 8 equations of degree 2 in 16 variables
--------------------------------------------------------------------------------
Perform Gaussian Elimination
--------------------------------------------------------------------------------
Total: 16 equations in 16 variables
8 linear equations in 16 variables
8 non-linear equations in 16 variables
-> 8 non-linear equations in 8 variables
8 equations in 8 variables
----------